# Question Answering with `HanaSparqlQAChain`

## Setup and Installation
To use this feature, install the `langchain-hana` package:

In [ ]:
pip install langchain_hana

And then, create a connection to your SAP HANA Cloud instance.

In [ ]:
import os

from dotenv import load_dotenv
from hdbcli import dbapi

# Load environment variables if needed
load_dotenv()

# Establish connection to SAP HANA Cloud
connection = dbapi.connect(
    address=os.environ.get("HANA_DB_ADDRESS"),
    port=os.environ.get("HANA_DB_PORT"),
    user=os.environ.get("HANA_DB_USER"),
    password=os.environ.get("HANA_DB_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=False,
)

`HanaSparqlQAChain` ties together:

1. **Schema-aware SPARQL generation**  
2. **Query execution** against SAP HANA  
3. **Natural-language answer formatting**


## Initialization

You need:

- An **LLM** to generate and interpret queries  
- A **`HanaRdfGraph`** (with connection, `graph_uri`, and ontology)

Follow the steps here [HanaRdfGraph](/docs/integrations/graphs/sap_hana_rdf_graph) to know more about creating a `HanaRdfGraph` instance.

Import the HanaSparqlQAChain

In [ ]:
from langchain_hana import HanaSparqlQAChain

In [ ]:
qa_chain = HanaSparqlQAChain.from_llm(
    llm=llm, graph=graph, allow_dangerous_requests=True, verbose=True
)

## Pipeline Overview

1. **SPARQL Generation**  
   - Uses `SPARQL_GENERATION_SELECT_PROMPT`  
   - Inputs:  
     - `schema` (Turtle from `graph.get_schema`)  
     - `prompt` (user’s question)  
2. **Query Post-processing**  
   - Extracts the SPARQL code from the llm output.
   - Inject `FROM <graph_uri>` if missing  
   - Ensure required common prefixes are declared (`rdf:`, `rdfs:`, `owl:`, `xsd:`)  
3. **Execution**  
   - Calls `graph.query(generated_sparql)`  
4. **Answer Formulation**  x
   - Uses `SPARQL_QA_PROMPT`  
   - Inputs:  
     - `context` (raw query results)  
     - `prompt` (original question)  

## Prompt Templates


#### "SPARQL Generation" prompt

The `sparql_generation_prompt` is used to guide the LLM in generating a SPARQL query from the user question and the provided schema.

#### Answering prompt

The `qa_prompt` instructs the LLM to create a natural language answer based solely on the database results.

The default prompts can be found here: [`prompts.py`](https://github.com/SAP/langchain-integration-for-sap-hana-cloud/blob/main/langchain_hana/chains/graph_qa/prompts.py)

## Customizing Prompts

You can override the defaults at initialization:

```python
qa_chain = HanaSparqlQAChain.from_llm(
    llm=llm,
    graph=graph,
    allow_dangerous_requests=True,
    verbose=True,
    sparql_generation_prompt=YOUR_SPARQL_PROMPT,
    qa_prompt=YOUR_QA_PROMPT
)
```

Or swap them afterward:

```python
qa_chain.sparql_generation_chain.prompt = YOUR_SPARQL_PROMPT
qa_chain.qa_chain.prompt              = YOUR_QA_PROMPT
```

> - `sparql_generation_prompt` must have the input variables: `["schema", "prompt"]`
> - `qa_prompt` must have the input variables: `["context", "prompt"]`

## Example: Question Answering over a “Movies” Knowledge Graph

**Prerequisite**:  
You must have an SAP HANA Cloud instance with the **triple store** feature enabled.  
For detailed instructions, refer to: [Enable Triple Store](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-knowledge-graph-guide/enable-triple-store/)<br />
Load the `kgdocu_movies` example data. See [Knowledge Graph Example](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-knowledge-graph-guide/knowledge-graph-example).

Below we’ll:

1. Instantiate the `HanaRdfGraph` pointing at our “movies” data graph  
2. Wrap it in a `HanaSparqlQAChain` powered by an LLM  
3. Ask natural-language questions and print out the chain’s responses  

This demonstrates how the LLM generates SPARQL under the hood, executes it against SAP HANA, and returns a human-readable answer.

We'll use the `sap-ai-sdk-gen` package. Currently still installed via: 

`pip install "generative-ai-hub-sdk[all]"` 

Please check [sap-ai-sdk-gen](https://pypi.org/project/sap-ai-sdk-gen/) for future releases.

First, create a connection to your SAP HANA Cloud instance.

In [ ]:
import os

from dotenv import load_dotenv
from hdbcli import dbapi

# Load environment variables if needed
load_dotenv()

# Establish connection to SAP HANA Cloud
connection = dbapi.connect(
    address=os.environ.get("HANA_DB_ADDRESS"),
    port=os.environ.get("HANA_DB_PORT"),
    user=os.environ.get("HANA_DB_USER"),
    password=os.environ.get("HANA_DB_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=False,
)

Then, set up the knowledge graph instance

In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from langchain_hana import HanaRdfGraph, HanaSparqlQAChain

# from langchain_openai import ChatOpenAI  # or your chosen LLM

In [ ]:
# Set up the Knowledge Graph
graph_uri = "kgdocu_movies"

graph = HanaRdfGraph(
    connection=connection, graph_uri=graph_uri, auto_extract_ontology=True
)

In [ ]:
# a basic graph schema is extracted from the data graph. This schema will guide the LLM to generate a proper SPARQL query.
print(graph.get_schema)

After that, initialise the LLM.

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(proxy_model_name="gpt-4o", temperature=0)

Then, we create a SPARQL QA Chain

In [ ]:
# Create a SPARQL QA Chain
chain = HanaSparqlQAChain.from_llm(
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    graph=graph,
)

In [ ]:
# output = chain.invoke("Which movies are in the data?")
# output = chain.invoke("In which movies did Keanu Reeves and Carrie-Anne Moss play in together")
# output = chain.invoke("which movie genres are in the data?")
# output = chain.invoke("which are the two most assigned movie genres?")
# output = chain.invoke("where were the actors of "Blade Runner" born?")
# output = chain.invoke("which actors acted together in a movie and were born in the same city?")
output = chain.invoke("which actors acted in Blade Runner?")

print(output["result"])

### What’s happening under the hood?

1. **SPARQL Generation**  
   The chain invokes the LLM with your Turtle-formatted ontology (`graph.get_schema`) and the user’s question using the `SPARQL_GENERATION_SELECT_PROMPT`. The LLM then emits a valid `SELECT` query tailored to your schema.

2. **Pre-processing & Execution**  
   - **Extract & clean**: Pull the raw SPARQL text out of the LLM’s response.  
   - **Inject graph context**: Add `FROM <graph_uri>` if it’s missing and ensure common prefixes (`rdf:`, `rdfs:`, `owl:`, `xsd:`) are declared.  
   - **Run on HANA**: Execute the finalized query via `HanaRdfGraph.query()` over your named graph.

3. **Answer Formulation**  
   The returned CSV (or Turtle) results feed into the LLM again—this time with the `SPARQL_QA_PROMPT`. The LLM produces a concise, human-readable answer strictly based on the retrieved data, without hallucination.
